In [1]:
import os
import pandas as pd
import glob

os.sys.path.insert(0, '../evaluation')

from evaluate import evaluate_all_systems

In [3]:
evaluate_all_systems(references_list=[[0, 1, 2], [0, 1]])

In [4]:
models = [os.path.basename(p) for p in glob.glob('../data/models/*')]

dfs = [pd.read_csv(f'../data/models/{model}/system_evaluation.csv', 
                   index_col=['subset', 'references', 'metric']) for model in models]

df = pd.concat(dfs, keys=models)

df.head()

value
      subset references metric       
abe-1 1size  [0, 1, 2]  bleu     58.5
                        bleu_1   89.0
                        bleu_2   69.8
                        bleu_3   54.3
                        bleu_4   43.2

# BLEU

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

Download multi-bleu.perl and put it in **../evaluation/tools/**

# METEOR

http://www.cs.cmu.edu/~alavie/METEOR/

Download meteor-1.5 and unpack into **../evaluation/tools/**

The jar path must be **../evaluation/tools/meteor-1.5/meteor-1.5.jar**

# TER

http://www.cs.umd.edu/~snover/tercom/

Download tercom-0.7.25 and unpack into **../evaluation/tools/**

The jar path must be **../evaluation/tools/tercom-0.7.25/tercom-0.7.25.jar**

<img src="../img/competition_results.jpg" />

## Melbourne

<a href="../data/models/melbourne/WebNLG Challenge - Bayu Distiawan T.pdf">report</a>

In [4]:
df.loc[('melbourne', slice(None), slice(None))].unstack()

value                             \
metric                       avg_n_stop_words avg_n_tokens   bleu bleu_1   
          subset  references                                               
melbourne 1size   [0, 1, 2]          0.252278     9.927313  63.81   87.5   
          2size   [0, 1, 2]          0.306310    16.598854  51.25   82.2   
          3size   [0, 1, 2]          0.319356    22.341969  43.42   80.3   
          4size   [0, 1, 2]          0.329409    28.953168  42.14   78.6   
          5size   [0, 1, 2]          0.353569    34.796226  36.72   75.3   
          6size   [0, 1, 2]          0.345861    43.958333  52.16   81.8   
          7size   [0, 1, 2]          0.362121    44.380952  48.65   81.4   
          all-cat [0, 1, 2]          0.308209    21.827068  45.13   79.9   
          new-cat [0, 1, 2]          0.306252    20.819304  33.27   72.2   
          old-cat [0, 1, 2]          0.310004    22.751802  54.52   86.3   

                                                                          \
metric                       bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
          subset  references                                               
melbourne 1size   [0, 1, 2]    72.5   60.5   49.9               0.253384   
          2size   [0, 1, 2]    59.7   44.8   34.0               0.301743   
          3size   [0, 1, 2]    54.7   37.4   25.6               0.317834   
          4size   [0, 1, 2]    52.7   35.6   24.6               0.326356   
          5size   [0, 1, 2]    47.9   29.7   18.4               0.352673   
          6size   [0, 1, 2]    60.3   45.7   35.3               0.349763   
          7size   [0, 1, 2]    60.3   44.7   33.6               0.364807   
          all-cat [0, 1, 2]    55.4   38.8   27.4               0.320407   
          new-cat [0, 1, 2]    44.0   27.1   16.2               0.324205   
          old-cat [0, 1, 2]    65.0   48.5   36.6               0.317219   

                                                  
metric                          meteor       ter  
          subset  references                      
melbourne 1size   [0, 1, 2]   0.471922  0.274120  
          2size   [0, 1, 2]   0.421393  0.389833  
          3size   [0, 1, 2]   0.376657  0.470233  
          4size   [0, 1, 2]   0.358887  0.520130  
          5size   [0, 1, 2]   0.327765  0.579703  
          6size   [0, 1, 2]   0.378531  0.469173  
          7size   [0, 1, 2]   0.355614  0.467217  
          all-cat [0, 1, 2]   0.376396  0.474403  
          new-cat [0, 1, 2]   0.332213  0.559212  
          old-cat [0, 1, 2]   0.414642  0.402598

In [5]:
!head -100 ../data/models/melbourne/melbourne.txt | tail -10

## Tilb-SMT

<a href="../data/models/tilburg-smt/report.pdf">report</a>

In [6]:
df.loc[('tilburg-smt', slice(None), slice(None))].unstack()

value                             \
metric                         avg_n_stop_words avg_n_tokens   bleu bleu_1   
            subset  references                                               
tilburg-smt 1size   [0, 1, 2]          0.188913     8.033040  52.16   88.4   
            2size   [0, 1, 2]          0.209839    15.985673  48.16   76.7   
            3size   [0, 1, 2]          0.217507    23.606218  43.42   73.2   
            4size   [0, 1, 2]          0.218908    31.418733  42.44   72.2   
            5size   [0, 1, 2]          0.234479    38.181132  38.68   71.2   
            6size   [0, 1, 2]          0.323156    49.208333  49.71   78.6   
            7size   [0, 1, 2]          0.313886    54.809524  44.44   75.8   
            all-cat [0, 1, 2]          0.214235    22.660043  44.28   74.4   
            new-cat [0, 1, 2]          0.125485    20.683502  29.88   65.0   
            old-cat [0, 1, 2]          0.295673    24.473738  54.29   81.8   

                                                                            \
metric                         bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
            subset  references                                               
tilburg-smt 1size   [0, 1, 2]    70.9   56.0   45.7               0.193858   
            2size   [0, 1, 2]    57.7   43.6   33.6               0.207026   
            3size   [0, 1, 2]    51.9   36.8   26.2               0.217845   
            4size   [0, 1, 2]    50.5   35.3   25.2               0.217098   
            5size   [0, 1, 2]    47.8   31.4   20.9               0.234631   
            6size   [0, 1, 2]    59.2   42.8   30.7               0.329382   
            7size   [0, 1, 2]    53.5   37.0   26.0               0.322328   
            all-cat [0, 1, 2]    53.0   37.5   26.8               0.224137   
            new-cat [0, 1, 2]    40.6   24.5   14.9               0.122253   
            old-cat [0, 1, 2]    62.6   47.4   35.8               0.303148   

                                                    
metric                            meteor       ter  
            subset  references                      
tilburg-smt 1size   [0, 1, 2]   0.412354  0.367070  
            2size   [0, 1, 2]   0.399773  0.464433  
            3size   [0, 1, 2]   0.380756  0.535855  
            4size   [0, 1, 2]   0.372728  0.573584  
            5size   [0, 1, 2]   0.361045  0.615279  
            6size   [0, 1, 2]   0.404062  0.565714  
            7size   [0, 1, 2]   0.390495  0.636231  
            all-cat [0, 1, 2]   0.380463  0.537692  
            new-cat [0, 1, 2]   0.333329  0.617078  
            old-cat [0, 1, 2]   0.421025  0.470480

In [7]:
!head -100 ../data/models/tilburg-smt/tilburg-smt.txt | tail -10

novel castle is english language 
. eric flint was born in burbank , california . 
macmillan publishers is the parent company of farrar , straus and giroux 
john cowper powys notablework a glastonbury romance 
soho press , united states . 
, published by faber and faber 
asian americans are an ethnic group . 
english is the language of the united states . 
weymouth sands was preceded by a glastonbury romance 
the manager of a.c. chievo verona is rolando maran 


## PKUWriter

<a href="../data/models/pkuwriter/PKUWriter_system.pdf">report</a>

In [8]:
df.loc[('pkuwriter', slice(None), slice(None))].unstack()

value                             \
metric                       avg_n_stop_words avg_n_tokens   bleu bleu_1   
          subset  references                                               
pkuwriter 1size   [0, 1, 2]          0.288924    10.788546  53.15   77.3   
          2size   [0, 1, 2]          0.304937    15.813754  45.34   75.4   
          3size   [0, 1, 2]          0.325455    21.528497  34.89   70.7   
          4size   [0, 1, 2]          0.339113    25.914601  36.80   73.1   
          5size   [0, 1, 2]          0.370858    30.664151  34.17   72.4   
          6size   [0, 1, 2]          0.371622    42.041667  52.89   84.1   
          7size   [0, 1, 2]          0.383537    47.333333  52.63   84.8   
          all-cat [0, 1, 2]          0.323077    20.549409  39.85   73.9   
          new-cat [0, 1, 2]          0.319450    18.693603  25.36   63.3   
          old-cat [0, 1, 2]          0.326405    22.252317  51.18   82.0   

                                                                          \
metric                       bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
          subset  references                                               
pkuwriter 1size   [0, 1, 2]    59.1   46.8   37.4               0.283585   
          2size   [0, 1, 2]    53.6   40.9   31.7               0.297699   
          3size   [0, 1, 2]    44.5   29.8   20.3               0.327196   
          4size   [0, 1, 2]    48.6   34.3   24.9               0.331136   
          5size   [0, 1, 2]    45.9   30.7   21.3               0.358110   
          6size   [0, 1, 2]    64.5   49.1   37.6               0.373637   
          7size   [0, 1, 2]    62.8   45.0   34.1               0.385312   
          all-cat [0, 1, 2]    49.9   35.6   26.0               0.327627   
          new-cat [0, 1, 2]    35.7   22.0   13.5               0.318204   
          old-cat [0, 1, 2]    60.7   45.9   35.4               0.334891   

                                                  
metric                          meteor       ter  
          subset  references                      
pkuwriter 1size   [0, 1, 2]   0.415371  0.371388  
          2size   [0, 1, 2]   0.355510  0.481268  
          3size   [0, 1, 2]   0.301129  0.602554  
          4size   [0, 1, 2]   0.292479  0.589683  
          5size   [0, 1, 2]   0.274453  0.623402  
          6size   [0, 1, 2]   0.389259  0.425865  
          7size   [0, 1, 2]   0.380657  0.469160  
          all-cat [0, 1, 2]   0.317300  0.553278  
          new-cat [0, 1, 2]   0.248942  0.674590  
          old-cat [0, 1, 2]   0.377063  0.450567

In [9]:
!head -100 ../data/models/pkuwriter/pkuwriter.txt | tail -10

the novel into battle is written by the english language .
john cowper powys was born in burbank , california .
the parent company of the macmillan publishers press is the macmillan publishers company of the macmillan publishers .
the novel owen glendower is a notable work by john cowper powys .
soho press is located in united states .
the polish academy for writtenwork is referencenumber _ in _ the _ national _ register _ of _ historic _ places .
asian americans are an ethnic group in the united states .
english is the language spoken in the united states .
the novel into battle is preceded by a glastonbury romance .
rolando maran manages the a . c . chievo verona .


## UPF-FORGe

<a href="../data/models/upf-forge/WebNLG_V0.2.pdf">report</a>

In [10]:
df.loc[('upf-forge', slice(None), slice(None))].unstack()

value                             \
metric                       avg_n_stop_words avg_n_tokens   bleu bleu_1   
          subset  references                                               
upf-forge 1size   [0, 1, 2]          0.236928    10.055066  60.29   86.5   
          2size   [0, 1, 2]          0.267511    17.954155  43.24   77.5   
          3size   [0, 1, 2]          0.288092    26.101036  36.84   74.8   
          4size   [0, 1, 2]          0.297504    34.876033  35.54   73.6   
          5size   [0, 1, 2]          0.312257    42.184906  32.66   72.4   
          6size   [0, 1, 2]          0.320526    52.416667  38.33   76.5   
          7size   [0, 1, 2]          0.354718    58.619048  37.84   75.7   
          all-cat [0, 1, 2]          0.278203    25.367347  38.65   75.5   
          new-cat [0, 1, 2]          0.252808    23.294052  35.70   74.2   
          old-cat [0, 1, 2]          0.301506    27.269825  40.88   76.4   

                                                                          \
metric                       bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
          subset  references                                               
upf-forge 1size   [0, 1, 2]    68.7   55.3   44.8               0.239430   
          2size   [0, 1, 2]    52.6   35.9   23.9               0.268273   
          3size   [0, 1, 2]    46.9   29.5   17.8               0.287742   
          4size   [0, 1, 2]    45.8   28.3   16.8               0.292259   
          5size   [0, 1, 2]    44.0   25.1   14.2               0.308078   
          6size   [0, 1, 2]    49.5   30.6   18.6               0.318760   
          7size   [0, 1, 2]    48.8   30.1   18.4               0.355808   
          all-cat [0, 1, 2]    48.7   31.1   19.5               0.289114   
          new-cat [0, 1, 2]    45.7   28.3   16.9               0.261672   
          old-cat [0, 1, 2]    51.1   33.3   21.5               0.310624   

                                                  
metric                          meteor       ter  
          subset  references                      
upf-forge 1size   [0, 1, 2]   0.469279  0.270624  
          2size   [0, 1, 2]   0.409700  0.467734  
          3size   [0, 1, 2]   0.383582  0.567642  
          4size   [0, 1, 2]   0.376883  0.622324  
          5size   [0, 1, 2]   0.367748  0.665456  
          6size   [0, 1, 2]   0.402804  0.598195  
          7size   [0, 1, 2]   0.399411  0.654687  
          all-cat [0, 1, 2]   0.391077  0.559327  
          new-cat [0, 1, 2]   0.373621  0.559565  
          old-cat [0, 1, 2]   0.405867  0.559126

In [11]:
!head -100 ../data/models/upf-forge/upf-forge.txt | tail -10

the english language is spoken in castle ( novel ) .
eric flint was born in burbank ( california ) .
the parent company of farrar , straus and giroux is macmillan publishers .
a glastonbury romance wrote john cowper powys .
soho press is published in the united states .
the publisher of the secret scripture is faber and faber .
a ethnic group of the united states are asian americans .
the english language is spoken in the united states .
weymouth sands follows a glastonbury romance .
rolando maran is the manager of a.c. chievo verona .


## Tilb-Pipeline

<a href="../data/models/tilburg-template/report.pdf">report</a>

In [12]:
df.loc[('tilburg-template', slice(None), slice(None))].unstack()

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
tilburg-template 1size   [0, 1, 2]          0.149068     6.420705  44.81   
                 2size   [0, 1, 2]          0.197241    11.300860  34.84   
                 3size   [0, 1, 2]          0.231148    16.373057  33.91   
                 4size   [0, 1, 2]          0.240871    21.415978  34.05   
                 5size   [0, 1, 2]          0.267942    28.279245  33.03   
                 6size   [0, 1, 2]          0.325806    44.250000  39.83   
                 7size   [0, 1, 2]          0.333492    52.476190  40.68   
                 all-cat [0, 1, 2]          0.214286    16.439850  35.29   
                 new-cat [0, 1, 2]          0.159253    10.469136  20.65   
                 old-cat [0, 1, 2]          0.264785    21.918641  44.34   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
tilburg-template 1size   [0, 1, 2]    90.8   72.5   56.6   43.6   
                 2size   [0, 1, 2]    85.2   61.8   44.5   32.4   
                 3size   [0, 1, 2]    82.0   56.3   38.3   26.3   
                 4size   [0, 1, 2]    81.3   55.8   38.6   26.7   
                 5size   [0, 1, 2]    78.2   51.0   33.0   21.1   
                 6size   [0, 1, 2]    77.8   51.3   33.5   21.1   
                 7size   [0, 1, 2]    77.3   52.0   32.9   20.7   
                 all-cat [0, 1, 2]    81.8   56.6   38.8   26.5   
                 new-cat [0, 1, 2]    81.8   57.1   39.3   26.6   
                 old-cat [0, 1, 2]    81.8   56.4   38.6   26.5   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
tilburg-template 1size   [0, 1, 2]                0.198285  0.323192  0.475854  
                 2size   [0, 1, 2]                0.242140  0.318193  0.515267  
                 3size   [0, 1, 2]                0.267722  0.307431  0.559991  
                 4size   [0, 1, 2]                0.271546  0.297620  0.578120  
                 5size   [0, 1, 2]                0.302108  0.296692  0.629880  
                 6size   [0, 1, 2]                0.326742  0.358336  0.571128  
                 7size   [0, 1, 2]                0.338475  0.387253  0.604177  
                 all-cat [0, 1, 2]                0.271798  0.308340  0.566023  
                 new-cat [0, 1, 2]                0.236921  0.216254  0.657518  
                 old-cat [0, 1, 2]                0.287084  0.382392  0.488559

In [13]:
!head -100 ../data/models/tilburg-template/tilburg-template.txt | tail -10

castle is in english . 
eric flint was born in burbank , california . 
macmillan publishers owns farrar , straus , and giroux . 
a glastonbury romance is a notable work by john cowper powys . 
soho press is in united states . 
the secret scripture publisher of faber and faber . 
asian americans live in united states . 
united states is in english . 
weymouth sands was preceded by a glastonbury romance . 
rolando maran manages a.c. chievo verona . 


## Tilb-NMT

<a href="../data/models/tilburg-nmt/report.pdf">report</a>

In [14]:
df.loc[('tilburg-nmt', slice(None), slice(None))].unstack()

value                             \
metric                         avg_n_stop_words avg_n_tokens   bleu bleu_1   
            subset  references                                               
tilburg-nmt 1size   [0, 1, 2]          0.276343    11.766520  50.33   74.2   
            2size   [0, 1, 2]          0.313078    19.693410  37.13   67.5   
            3size   [0, 1, 2]          0.313660    26.632124  31.85   66.4   
            4size   [0, 1, 2]          0.304591    31.162534  32.92   68.9   
            5size   [0, 1, 2]          0.315162    36.649057  28.64   67.9   
            6size   [0, 1, 2]          0.329590    45.166667  39.88   73.8   
            7size   [0, 1, 2]          0.346100    47.714286  34.26   75.3   
            all-cat [0, 1, 2]          0.303469    24.492481  34.60   68.8   
            new-cat [0, 1, 2]          0.286198    24.894501  25.12   59.3   
            old-cat [0, 1, 2]          0.319317    24.123584  43.28   77.7   

                                                                            \
metric                         bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
            subset  references                                               
tilburg-nmt 1size   [0, 1, 2]    56.6   44.2   34.5               0.276675   
            2size   [0, 1, 2]    44.4   30.5   20.8               0.305834   
            3size   [0, 1, 2]    39.9   25.0   15.5               0.308463   
            4size   [0, 1, 2]    41.5   25.8   16.0               0.298091   
            5size   [0, 1, 2]    37.9   21.7   12.1               0.313324   
            6size   [0, 1, 2]    48.8   32.6   21.9               0.330258   
            7size   [0, 1, 2]    44.4   26.9   16.7               0.349301   
            all-cat [0, 1, 2]    42.7   27.5   17.7               0.304221   
            new-cat [0, 1, 2]    32.9   19.2   10.6               0.286822   
            old-cat [0, 1, 2]    52.1   35.5   24.4               0.320697   

                                                    
metric                            meteor       ter  
            subset  references                      
tilburg-nmt 1size   [0, 1, 2]   0.447765  0.429996  
            2size   [0, 1, 2]   0.384220  0.584420  
            3size   [0, 1, 2]   0.345341  0.646517  
            4size   [0, 1, 2]   0.330702  0.622947  
            5size   [0, 1, 2]   0.306137  0.665970  
            6size   [0, 1, 2]   0.346942  0.585564  
            7size   [0, 1, 2]   0.338627  0.616804  
            all-cat [0, 1, 2]   0.348784  0.609501  
            new-cat [0, 1, 2]   0.310575  0.721628  
            old-cat [0, 1, 2]   0.382624  0.514568

In [15]:
!head -100 ../data/models/tilburg-nmt/tilburg-nmt.txt | tail -10

english is the language of castle . 
eric flint was born in burbank , california . 
macmillan publishers is the parent company of farrar , straus , and giroux . 
a glastonbury romance is a notable work by john cowper powys . 
soho press is located in united states . 
th secret scriptures was published by faber and faber . 
asian americans are an ethnic group in united states . 
english is the language of united states . 
weymouth sands was preceded by a glastonbury romance . 
the manager of a.c. chievo verona is rolando maran . 


## Baseline

In [16]:
df.loc[('baseline_sorted', slice(None), slice(None))].unstack()

value                      \
metric                             avg_n_stop_words avg_n_tokens   bleu   
                subset  references                                        
baseline_sorted 1size   [0, 1, 2]          0.354087     9.885463  42.36   
                2size   [0, 1, 2]          0.378686    13.925501  33.83   
                3size   [0, 1, 2]          0.391894    19.044041  30.46   
                4size   [0, 1, 2]          0.389699    24.796143  31.11   
                5size   [0, 1, 2]          0.420930    29.543396  28.88   
                6size   [0, 1, 2]          0.363222    39.083333  53.25   
                7size   [0, 1, 2]          0.379823    40.238095  53.81   
                all-cat [0, 1, 2]          0.383399    18.964554  33.24   
                new-cat [0, 1, 2]          0.426870    14.810325   6.13   
                old-cat [0, 1, 2]          0.343509    22.776519  52.39   

                                                                \
metric                             bleu_1 bleu_2 bleu_3 bleu_4   
                subset  references                               
baseline_sorted 1size   [0, 1, 2]    65.1   45.2   38.8   33.6   
                2size   [0, 1, 2]    65.2   42.1   33.2   26.9   
                3size   [0, 1, 2]    64.3   40.1   29.5   22.2   
                4size   [0, 1, 2]    64.1   40.8   30.1   23.2   
                5size   [0, 1, 2]    63.2   38.9   26.7   19.8   
                6size   [0, 1, 2]    85.7   67.1   51.9   41.1   
                7size   [0, 1, 2]    89.9   69.3   54.3   43.1   
                all-cat [0, 1, 2]    65.4   42.3   31.8   24.8   
                new-cat [0, 1, 2]    38.9   10.1    4.9    2.8   
                old-cat [0, 1, 2]    81.2   61.1   47.1   36.8   

                                                                               
metric                             macro_avg_n_stop_words    meteor       ter  
                subset  references                                             
baseline_sorted 1size   [0, 1, 2]                0.345365  0.277388  0.491072  
                2size   [0, 1, 2]                0.367284  0.241359  0.575508  
                3size   [0, 1, 2]                0.385390  0.222312  0.639728  
                4size   [0, 1, 2]                0.373292  0.221627  0.653365  
                5size   [0, 1, 2]                0.419083  0.210370  0.669054  
                6size   [0, 1, 2]                0.368870  0.368557  0.426767  
                7size   [0, 1, 2]                0.381065  0.378776  0.402137  
                all-cat [0, 1, 2]                0.381655  0.235436  0.613080  
                new-cat [0, 1, 2]                0.428766  0.075353  0.807292  
                old-cat [0, 1, 2]                0.353545  0.377720  0.448648

In [17]:
!head -100 ../data/models/baseline_sorted/baseline_sorted.txt | tail -10

the english language is spoken in the republic of ireland .
author was born in burbank , california .
the parent company of wiley - blackwell is macmillan publishers .
a glastonbury romance was written by author .
lahore is in the united states .
the parent company of wiley - blackwell is parentcompany .
asian americans are an ethnic group in the united states .
english is the language of the united states .
a glastonbury romance was preceded by precededby .
the manager of a . c . chievo verona is rolando maran .


## Adapt

<a href="../data/models/adaptcentre/WebNLG_system.pdf">report</a>

In [18]:
df.loc[('adaptcentre', slice(None), slice(None))].unstack()

value                             \
metric                         avg_n_stop_words avg_n_tokens   bleu bleu_1   
            subset  references                                               
adaptcentre 1size   [0, 1, 2]          0.321420    11.618943  48.13   69.6   
            2size   [0, 1, 2]          0.366955    22.183381  33.77   58.3   
            3size   [0, 1, 2]          0.378001    35.129534  27.78   52.1   
            4size   [0, 1, 2]          0.373533    45.988981  27.56   50.3   
            5size   [0, 1, 2]          0.388933    54.207547  26.35   50.8   
            6size   [0, 1, 2]          0.369446    45.083333  62.03   86.5   
            7size   [0, 1, 2]          0.370930    48.047619  62.96   88.7   
            all-cat [0, 1, 2]          0.362630    32.076799  31.06   54.9   
            new-cat [0, 1, 2]          0.387794    40.300786  10.53   34.5   
            old-cat [0, 1, 2]          0.339538    24.530381  60.59   85.6   

                                                                            \
metric                         bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
            subset  references                                               
adaptcentre 1size   [0, 1, 2]    51.3   42.2   35.5               0.323791   
            2size   [0, 1, 2]    37.8   27.9   21.2               0.375872   
            3size   [0, 1, 2]    32.5   22.2   15.8               0.389602   
            4size   [0, 1, 2]    32.0   22.2   16.1               0.384090   
            5size   [0, 1, 2]    31.4   20.8   14.6               0.398120   
            6size   [0, 1, 2]    69.8   56.4   45.6               0.372458   
            7size   [0, 1, 2]    71.5   56.0   44.4               0.372646   
            all-cat [0, 1, 2]    35.7   25.3   18.7               0.381921   
            new-cat [0, 1, 2]    14.7    7.1    3.4               0.404367   
            old-cat [0, 1, 2]    67.9   53.7   43.1               0.348083   

                                                    
metric                            meteor       ter  
            subset  references                      
adaptcentre 1size   [0, 1, 2]   0.378119  0.442952  
            2size   [0, 1, 2]   0.326372  0.727348  
            3size   [0, 1, 2]   0.306462  0.935833  
            4size   [0, 1, 2]   0.296947  0.982271  
            5size   [0, 1, 2]   0.290825  0.996950  
            6size   [0, 1, 2]   0.432208  0.370827  
            7size   [0, 1, 2]   0.435427  0.372997  
            all-cat [0, 1, 2]   0.315980  0.849266  
            new-cat [0, 1, 2]   0.192333  1.404981  
            old-cat [0, 1, 2]   0.445389  0.378763

In [19]:
!head -100 ../data/models/adaptcentre/adaptcentre.txt | tail -10

the novel castle is the language spoken in the english language .
eric flint was born in burbank , california .
macmillan publishers is the parent company of farrar , straus and giroux .
a glastonbury romance is the notable work of john cowper powys .
soho press is located in the united states .
the novel the secret scriptures , is published by faber and faber .
asian americans are an ethnic group in the u.s .
english is the language of the united states .
a glastonbury romance is the sequel to weymouth sands .
the manager of a.c . chievo verona is rolando maran .


## UIT-VNU

<a href="../data/models/uit-vietnam/Phong-Nguyen-Tran_Dang-Tuan-Nguyen.docx">report</a>

In [20]:
df.loc[('uit-vietnam', slice(None), slice(None))].unstack()

value                             \
metric                         avg_n_stop_words avg_n_tokens   bleu bleu_1   
            subset  references                                               
uit-vietnam 1size   [0, 1, 2]          0.234653     9.557269  44.65   70.7   
            2size   [0, 1, 2]          0.115245     7.252149  16.52   68.6   
            3size   [0, 1, 2]          0.065954     5.891192   5.69   69.2   
            4size   [0, 1, 2]          0.027495     3.746556   0.31   61.3   
            5size   [0, 1, 2]          0.012361     2.532075   0.01   54.5   
            6size   [0, 1, 2]          0.000000     1.000000   0.00    0.0   
            7size   [0, 1, 2]          0.000000     1.000000   0.00    0.0   
            all-cat [0, 1, 2]          0.099607     6.025779   7.07   67.5   
            new-cat [0, 1, 2]          0.044283     2.519641   0.11   49.6   
            old-cat [0, 1, 2]          0.150373     9.243048  19.87   72.0   

                                                                            \
metric                         bleu_2 bleu_3 bleu_4 macro_avg_n_stop_words   
            subset  references                                               
uit-vietnam 1size   [0, 1, 2]    52.4   38.1   28.2               0.322885   
            2size   [0, 1, 2]    53.4   38.8   29.1               0.322007   
            3size   [0, 1, 2]    56.9   40.4   29.3               0.303870   
            4size   [0, 1, 2]    57.9   43.1   32.9               0.247059   
            5size   [0, 1, 2]    59.9   42.2   29.3               0.195231   
            6size   [0, 1, 2]     0.0    0.0    0.0               0.000000   
            7size   [0, 1, 2]     0.0    0.0    0.0               0.000000   
            all-cat [0, 1, 2]    54.4   39.5   29.2               0.300713   
            new-cat [0, 1, 2]    53.0   37.6   26.3               0.214254   
            old-cat [0, 1, 2]    54.7   39.8   29.7               0.322340   

                                                    
metric                            meteor       ter  
            subset  references                      
uit-vietnam 1size   [0, 1, 2]   0.328469  0.625150  
            2size   [0, 1, 2]   0.156637  0.764153  
            3size   [0, 1, 2]   0.091948  0.831528  
            4size   [0, 1, 2]   0.042366  0.866558  
            5size   [0, 1, 2]   0.021128  0.896699  
            6size   [0, 1, 2]   0.000000  0.929323  
            7size   [0, 1, 2]   0.000000  0.929577  
            all-cat [0, 1, 2]   0.099207  0.827492  
            new-cat [0, 1, 2]   0.033486  0.877597  
            old-cat [0, 1, 2]   0.153499  0.785071

In [21]:
!head -100 ../data/models/uit-vietnam/uit-vietnam.txt | tail -10

# All results

## 3 first references

In [5]:
df.loc[(slice(None), 'all-cat', '[0, 1, 2]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
BIU_nmt          all-cat [0, 1, 2]          0.342847    24.142320  46.37   
abe-2            all-cat [0, 1, 2]          0.253881    22.243287  45.42   
melbourne        all-cat [0, 1, 2]          0.308209    21.827068  45.13   
tilburg-smt      all-cat [0, 1, 2]          0.214235    22.660043  44.28   
abe-1            all-cat [0, 1, 2]          0.229932    25.898496  41.90   
pkuwriter        all-cat [0, 1, 2]          0.323077    20.549409  39.85   
upf-forge        all-cat [0, 1, 2]          0.278203    25.367347  38.65   
tilburg-template all-cat [0, 1, 2]          0.214286    16.439850  35.29   
tilburg-nmt      all-cat [0, 1, 2]          0.303469    24.492481  34.60   
baseline_sorted  all-cat [0, 1, 2]          0.383399    18.964554  33.24   
adaptcentre      all-cat [0, 1, 2]          0.362630    32.076799  31.06   
uit-vietnam      all-cat [0, 1, 2]          0.099607     6.025779   7.07   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
BIU_nmt          all-cat [0, 1, 2]    78.4   55.3   38.8   27.5   
abe-2            all-cat [0, 1, 2]    80.8   56.3   38.7   26.6   
melbourne        all-cat [0, 1, 2]    79.9   55.4   38.8   27.4   
tilburg-smt      all-cat [0, 1, 2]    74.4   53.0   37.5   26.8   
abe-1            all-cat [0, 1, 2]    71.4   50.3   35.1   24.5   
pkuwriter        all-cat [0, 1, 2]    73.9   49.9   35.6   26.0   
upf-forge        all-cat [0, 1, 2]    75.5   48.7   31.1   19.5   
tilburg-template all-cat [0, 1, 2]    81.8   56.6   38.8   26.5   
tilburg-nmt      all-cat [0, 1, 2]    68.8   42.7   27.5   17.7   
baseline_sorted  all-cat [0, 1, 2]    65.4   42.3   31.8   24.8   
adaptcentre      all-cat [0, 1, 2]    54.9   35.7   25.3   18.7   
uit-vietnam      all-cat [0, 1, 2]    67.5   54.4   39.5   29.2   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
BIU_nmt          all-cat [0, 1, 2]                0.331007  0.391809  0.488673  
abe-2            all-cat [0, 1, 2]                0.236695  0.402532  0.475026  
melbourne        all-cat [0, 1, 2]                0.320407  0.376396  0.474403  
tilburg-smt      all-cat [0, 1, 2]                0.224137  0.380463  0.537692  
abe-1            all-cat [0, 1, 2]                0.222895  0.398795  0.573897  
pkuwriter        all-cat [0, 1, 2]                0.327627  0.317300  0.553278  
upf-forge        all-cat [0, 1, 2]                0.289114  0.391077  0.559327  
tilburg-template all-cat [0, 1, 2]                0.271798  0.308340  0.566023  
tilburg-nmt      all-cat [0, 1, 2]                0.304221  0.348784  0.609501  
baseline_sorted  all-cat [0, 1, 2]                0.381655  0.235436  0.613080  
adaptcentre      all-cat [0, 1, 2]                0.381921  0.315980  0.849266  
uit-vietnam      all-cat [0, 1, 2]                0.300713  0.099207  0.827492

In [6]:
df.loc[(slice(None), 'old-cat', '[0, 1, 2]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
adaptcentre      old-cat [0, 1, 2]          0.339538    24.530381  60.59   
melbourne        old-cat [0, 1, 2]          0.310004    22.751802  54.52   
tilburg-smt      old-cat [0, 1, 2]          0.295673    24.473738  54.29   
BIU_nmt          old-cat [0, 1, 2]          0.338298    24.892894  53.63   
baseline_sorted  old-cat [0, 1, 2]          0.343509    22.776519  52.39   
abe-2            old-cat [0, 1, 2]          0.328465    24.822863  51.77   
pkuwriter        old-cat [0, 1, 2]          0.326405    22.252317  51.18   
abe-1            old-cat [0, 1, 2]          0.313532    27.391349  48.41   
tilburg-template old-cat [0, 1, 2]          0.264785    21.918641  44.34   
tilburg-nmt      old-cat [0, 1, 2]          0.319317    24.123584  43.28   
upf-forge        old-cat [0, 1, 2]          0.301506    27.269825  40.88   
uit-vietnam      old-cat [0, 1, 2]          0.150373     9.243048  19.87   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
adaptcentre      old-cat [0, 1, 2]    85.6   67.9   53.7   43.1   
melbourne        old-cat [0, 1, 2]    86.3   65.0   48.5   36.6   
tilburg-smt      old-cat [0, 1, 2]    81.8   62.6   47.4   35.8   
BIU_nmt          old-cat [0, 1, 2]    83.2   62.3   46.0   34.7   
baseline_sorted  old-cat [0, 1, 2]    81.2   61.1   47.1   36.8   
abe-2            old-cat [0, 1, 2]    82.4   60.6   44.3   32.5   
pkuwriter        old-cat [0, 1, 2]    82.0   60.7   45.9   35.4   
abe-1            old-cat [0, 1, 2]    76.1   56.3   41.6   30.8   
tilburg-template old-cat [0, 1, 2]    81.8   56.4   38.6   26.5   
tilburg-nmt      old-cat [0, 1, 2]    77.7   52.1   35.5   24.4   
upf-forge        old-cat [0, 1, 2]    76.4   51.1   33.3   21.5   
uit-vietnam      old-cat [0, 1, 2]    72.0   54.7   39.8   29.7   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
adaptcentre      old-cat [0, 1, 2]                0.348083  0.445389  0.378763  
melbourne        old-cat [0, 1, 2]                0.317219  0.414642  0.402598  
tilburg-smt      old-cat [0, 1, 2]                0.303148  0.421025  0.470480  
BIU_nmt          old-cat [0, 1, 2]                0.329996  0.417234  0.434748  
baseline_sorted  old-cat [0, 1, 2]                0.353545  0.377720  0.448648  
abe-2            old-cat [0, 1, 2]                0.322999  0.425242  0.446388  
pkuwriter        old-cat [0, 1, 2]                0.334891  0.377063  0.450567  
abe-1            old-cat [0, 1, 2]                0.311334  0.423174  0.524333  
tilburg-template old-cat [0, 1, 2]                0.287084  0.382392  0.488559  
tilburg-nmt      old-cat [0, 1, 2]                0.320697  0.382624  0.514568  
upf-forge        old-cat [0, 1, 2]                0.310624  0.405867  0.559126  
uit-vietnam      old-cat [0, 1, 2]                0.322340  0.153499  0.785071

In [7]:
df.loc[(slice(None), 'new-cat', '[0, 1, 2]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
BIU_nmt          new-cat [0, 1, 2]          0.348138    23.324355  37.55   
upf-forge        new-cat [0, 1, 2]          0.252808    23.294052  35.70   
abe-2            new-cat [0, 1, 2]          0.150052    19.432099  35.13   
abe-1            new-cat [0, 1, 2]          0.127116    24.271605  33.48   
melbourne        new-cat [0, 1, 2]          0.306252    20.819304  33.27   
tilburg-smt      new-cat [0, 1, 2]          0.125485    20.683502  29.88   
pkuwriter        new-cat [0, 1, 2]          0.319450    18.693603  25.36   
tilburg-nmt      new-cat [0, 1, 2]          0.286198    24.894501  25.12   
tilburg-template new-cat [0, 1, 2]          0.159253    10.469136  20.65   
adaptcentre      new-cat [0, 1, 2]          0.387794    40.300786  10.53   
baseline_sorted  new-cat [0, 1, 2]          0.426870    14.810325   6.13   
uit-vietnam      new-cat [0, 1, 2]          0.044283     2.519641   0.11   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
BIU_nmt          new-cat [0, 1, 2]    72.8   47.1   30.2   19.2   
upf-forge        new-cat [0, 1, 2]    74.2   45.7   28.3   16.9   
abe-2            new-cat [0, 1, 2]    78.5   50.2   30.7   18.0   
abe-1            new-cat [0, 1, 2]    65.5   42.9   27.1   16.5   
melbourne        new-cat [0, 1, 2]    72.2   44.0   27.1   16.2   
tilburg-smt      new-cat [0, 1, 2]    65.0   40.6   24.5   14.9   
pkuwriter        new-cat [0, 1, 2]    63.3   35.7   22.0   13.5   
tilburg-nmt      new-cat [0, 1, 2]    59.3   32.9   19.2   10.6   
tilburg-template new-cat [0, 1, 2]    81.8   57.1   39.3   26.6   
adaptcentre      new-cat [0, 1, 2]    34.5   14.7    7.1    3.4   
baseline_sorted  new-cat [0, 1, 2]    38.9   10.1    4.9    2.8   
uit-vietnam      new-cat [0, 1, 2]    49.6   53.0   37.6   26.3   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
BIU_nmt          new-cat [0, 1, 2]                0.332109  0.362192  0.552363  
upf-forge        new-cat [0, 1, 2]                0.261672  0.373621  0.559565  
abe-2            new-cat [0, 1, 2]                0.142641  0.375496  0.508851  
abe-1            new-cat [0, 1, 2]                0.126515  0.370582  0.632438  
melbourne        new-cat [0, 1, 2]                0.324205  0.332213  0.559212  
tilburg-smt      new-cat [0, 1, 2]                0.122253  0.333329  0.617078  
pkuwriter        new-cat [0, 1, 2]                0.318204  0.248942  0.674590  
tilburg-nmt      new-cat [0, 1, 2]                0.286822  0.310575  0.721628  
tilburg-template new-cat [0, 1, 2]                0.236921  0.216254  0.657518  
adaptcentre      new-cat [0, 1, 2]                0.404367  0.192333  1.404981  
baseline_sorted  new-cat [0, 1, 2]                0.428766  0.075353  0.807292  
uit-vietnam      new-cat [0, 1, 2]                0.214254  0.033486  0.877597

## 2 first references

In [8]:
df.loc[(slice(None), 'all-cat', '[0, 1]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
BIU_nmt          all-cat [0, 1]             0.342847    24.142320  42.42   
abe-2            all-cat [0, 1]             0.253881    22.243287  41.55   
melbourne        all-cat [0, 1]             0.320407    21.827068  40.86   
tilburg-smt      all-cat [0, 1]             0.224137    22.660043  40.31   
abe-1            all-cat [0, 1]             0.229932    25.898496  38.57   
pkuwriter        all-cat [0, 1]             0.327627    20.549409  35.72   
upf-forge        all-cat [0, 1]             0.289114    25.367347  35.23   
tilburg-nmt      all-cat [0, 1]             0.304221    24.492481  31.88   
tilburg-template all-cat [0, 1]             0.271798    16.439850  29.88   
baseline_sorted  all-cat [0, 1]             0.381655    18.964554  29.38   
adaptcentre      all-cat [0, 1]             0.381921    32.076799  28.35   
uit-vietnam      all-cat [0, 1]             0.300713     6.025779   3.20   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
BIU_nmt          all-cat [0, 1]       75.7   51.1   34.8   24.1   
abe-2            all-cat [0, 1]       78.3   52.6   35.2   23.6   
melbourne        all-cat [0, 1]       77.3   51.5   35.0   24.0   
tilburg-smt      all-cat [0, 1]       72.1   49.5   33.8   23.5   
abe-1            all-cat [0, 1]       68.9   46.7   31.8   21.6   
pkuwriter        all-cat [0, 1]       71.5   46.5   32.5   23.2   
upf-forge        all-cat [0, 1]       72.5   44.9   27.8   17.0   
tilburg-nmt      all-cat [0, 1]       66.6   39.7   24.9   15.7   
tilburg-template all-cat [0, 1]       79.4   52.5   34.8   23.2   
baseline_sorted  all-cat [0, 1]       63.1   39.2   28.7   21.9   
adaptcentre      all-cat [0, 1]       52.5   32.9   22.8   16.5   
uit-vietnam      all-cat [0, 1]       65.3   50.9   35.9   25.9   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
BIU_nmt          all-cat [0, 1]                   0.331007  0.382362  0.509999  
abe-2            all-cat [0, 1]                   0.236695  0.393197  0.495541  
melbourne        all-cat [0, 1]                   0.308209  0.366541  0.497150  
tilburg-smt      all-cat [0, 1]                   0.214235  0.370970  0.558130  
abe-1            all-cat [0, 1]                   0.222895  0.389508  0.591344  
pkuwriter        all-cat [0, 1]                   0.323077  0.310660  0.572174  
upf-forge        all-cat [0, 1]                   0.278203  0.382659  0.578311  
tilburg-nmt      all-cat [0, 1]                   0.303469  0.340662  0.627890  
tilburg-template all-cat [0, 1]                   0.214286  0.300199  0.591137  
baseline_sorted  all-cat [0, 1]                   0.383399  0.229955  0.634349  
adaptcentre      all-cat [0, 1]                   0.362630  0.308486  0.867903  
uit-vietnam      all-cat [0, 1]                   0.099607  0.096917  0.853997

In [9]:
df.loc[(slice(None), 'old-cat', '[0, 1]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
adaptcentre      old-cat [0, 1]             0.348083    24.530381  55.96   
tilburg-smt      old-cat [0, 1]             0.303148    24.473738  49.95   
melbourne        old-cat [0, 1]             0.317219    22.751802  49.73   
BIU_nmt          old-cat [0, 1]             0.338298    24.892894  49.25   
abe-2            old-cat [0, 1]             0.328465    24.822863  48.19   
baseline_sorted  old-cat [0, 1]             0.353545    22.776519  47.88   
pkuwriter        old-cat [0, 1]             0.334891    22.252317  46.79   
abe-1            old-cat [0, 1]             0.313532    27.391349  44.90   
tilburg-template old-cat [0, 1]             0.287084    21.918641  40.28   
tilburg-nmt      old-cat [0, 1]             0.320697    24.123584  40.07   
upf-forge        old-cat [0, 1]             0.310624    27.269825  37.59   
uit-vietnam      old-cat [0, 1]             0.322340     9.243048  10.34   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
adaptcentre      old-cat [0, 1]       83.3   63.4   48.7   38.1   
tilburg-smt      old-cat [0, 1]       79.4   58.2   42.7   31.5   
melbourne        old-cat [0, 1]       83.8   60.6   44.0   32.2   
BIU_nmt          old-cat [0, 1]       80.7   57.8   41.4   30.4   
abe-2            old-cat [0, 1]       80.1   56.8   40.6   29.2   
baseline_sorted  old-cat [0, 1]       78.9   56.9   42.7   32.6   
pkuwriter        old-cat [0, 1]       79.9   57.0   42.2   31.8   
abe-1            old-cat [0, 1]       73.8   52.6   38.0   27.6   
tilburg-template old-cat [0, 1]       79.6   52.7   35.1   23.6   
tilburg-nmt      old-cat [0, 1]       75.6   48.6   32.3   21.8   
upf-forge        old-cat [0, 1]       73.7   47.4   30.1   19.0   
uit-vietnam      old-cat [0, 1]       69.7   51.0   36.0   26.2   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
adaptcentre      old-cat [0, 1]                   0.339538  0.434820  0.403129  
tilburg-smt      old-cat [0, 1]                   0.295673  0.411680  0.492699  
melbourne        old-cat [0, 1]                   0.310004  0.405084  0.425362  
BIU_nmt          old-cat [0, 1]                   0.329996  0.408112  0.456777  
abe-2            old-cat [0, 1]                   0.322999  0.417040  0.464132  
baseline_sorted  old-cat [0, 1]                   0.343509  0.369529  0.469063  
pkuwriter        old-cat [0, 1]                   0.326405  0.369948  0.467703  
abe-1            old-cat [0, 1]                   0.311334  0.414788  0.542564  
tilburg-template old-cat [0, 1]                   0.264785  0.373791  0.508895  
tilburg-nmt      old-cat [0, 1]                   0.319317  0.374744  0.535294  
upf-forge        old-cat [0, 1]                   0.301506  0.398648  0.579336  
uit-vietnam      old-cat [0, 1]                   0.150373  0.149876  0.807575

In [10]:
df.loc[(slice(None), 'new-cat', '[0, 1]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value                      \
metric                              avg_n_stop_words avg_n_tokens   bleu   
                 subset  references                                        
BIU_nmt          new-cat [0, 1]             0.348138    23.324355  34.11   
upf-forge        new-cat [0, 1]             0.261672    23.294052  32.11   
abe-2            new-cat [0, 1]             0.150052    19.432099  31.38   
abe-1            new-cat [0, 1]             0.127116    24.271605  30.36   
melbourne        new-cat [0, 1]             0.324205    20.819304  29.75   
tilburg-smt      new-cat [0, 1]             0.122253    20.683502  27.03   
tilburg-nmt      new-cat [0, 1]             0.286822    24.894501  22.92   
pkuwriter        new-cat [0, 1]             0.318204    18.693603  21.94   
tilburg-template new-cat [0, 1]             0.236921    10.469136  14.80   
adaptcentre      new-cat [0, 1]             0.404367    40.300786   9.21   
baseline_sorted  new-cat [0, 1]             0.428766    14.810325   4.84   
uit-vietnam      new-cat [0, 1]             0.214254     2.519641   0.03   

                                                                 \
metric                              bleu_1 bleu_2 bleu_3 bleu_4   
                 subset  references                               
BIU_nmt          new-cat [0, 1]       69.9   43.4   26.9   16.6   
upf-forge        new-cat [0, 1]       70.9   41.8   24.9   14.4   
abe-2            new-cat [0, 1]       75.8   46.6   27.5   15.6   
abe-1            new-cat [0, 1]       62.9   39.5   24.1   14.2   
melbourne        new-cat [0, 1]       69.6   40.7   24.2   14.1   
tilburg-smt      new-cat [0, 1]       62.7   38.0   22.1   13.0   
tilburg-nmt      new-cat [0, 1]       57.1   30.4   17.1    9.3   
pkuwriter        new-cat [0, 1]       60.6   32.8   19.7   11.6   
tilburg-template new-cat [0, 1]       79.0   52.1   34.1   22.1   
adaptcentre      new-cat [0, 1]       32.1   12.9    6.1    2.8   
baseline_sorted  new-cat [0, 1]       36.7    8.9    4.1    2.2   
uit-vietnam      new-cat [0, 1]       48.1   50.1   35.1   24.1   

                                                                                
metric                              macro_avg_n_stop_words    meteor       ter  
                 subset  references                                             
BIU_nmt          new-cat [0, 1]                   0.332109  0.352540  0.572651  
upf-forge        new-cat [0, 1]                   0.252808  0.363954  0.577105  
abe-2            new-cat [0, 1]                   0.142641  0.365071  0.532516  
abe-1            new-cat [0, 1]                   0.126515  0.360507  0.648768  
melbourne        new-cat [0, 1]                   0.306252  0.322382  0.581659  
tilburg-smt      new-cat [0, 1]                   0.125485  0.324028  0.635156  
tilburg-nmt      new-cat [0, 1]                   0.286198  0.302276  0.736895  
pkuwriter        new-cat [0, 1]                   0.319450  0.242964  0.695158  
tilburg-template new-cat [0, 1]                   0.159253  0.208912  0.687952  
adaptcentre      new-cat [0, 1]                   0.387794  0.187025  1.415038  
baseline_sorted  new-cat [0, 1]                   0.426870  0.072573  0.828925  
uit-vietnam      new-cat [0, 1]                   0.044283  0.032811  0.908645

# with template

In [11]:
df.loc[(slice(None), 'with-template', '[0, 1, 2]'), :].unstack().sort_values(('value', 'bleu'), ascending=False)

value               \
metric                                    avg_n_stop_words avg_n_tokens   
                 subset        references                                 
adaptcentre      with-template [0, 1, 2]          0.339990    19.000000   
melbourne        with-template [0, 1, 2]          0.304047    17.710145   
BIU_nmt          with-template [0, 1, 2]          0.328694    19.216074   
tilburg-smt      with-template [0, 1, 2]          0.294589    17.750988   
abe-1            with-template [0, 1, 2]          0.307357    18.749671   
abe-2            with-template [0, 1, 2]          0.307379    18.748353   
pkuwriter        with-template [0, 1, 2]          0.321128    18.027668   
baseline_sorted  with-template [0, 1, 2]          0.345936    17.911726   
tilburg-nmt      with-template [0, 1, 2]          0.313091    19.243742   
tilburg-template with-template [0, 1, 2]          0.264961    15.961792   
upf-forge        with-template [0, 1, 2]          0.298248    20.378129   
uit-vietnam      with-template [0, 1, 2]          0.333201    13.234519   

                                                                              \
metric                                      bleu bleu_1 bleu_2 bleu_3 bleu_4   
                 subset        references                                      
adaptcentre      with-template [0, 1, 2]   61.46   85.5   68.0   54.8   44.8   
melbourne        with-template [0, 1, 2]   59.15   88.9   69.4   54.1   42.4   
BIU_nmt          with-template [0, 1, 2]   58.47   86.1   66.5   51.2   39.9   
tilburg-smt      with-template [0, 1, 2]   56.04   85.2   66.0   50.8   39.4   
abe-1            with-template [0, 1, 2]   55.03   84.3   63.1   47.6   36.2   
abe-2            with-template [0, 1, 2]   55.03   84.3   63.1   47.6   36.2   
pkuwriter        with-template [0, 1, 2]   53.46   81.0   60.9   47.3   37.2   
baseline_sorted  with-template [0, 1, 2]   53.45   81.0   60.9   48.1   38.5   
tilburg-nmt      with-template [0, 1, 2]   48.61   80.3   56.7   41.0   29.9   
tilburg-template with-template [0, 1, 2]   45.86   85.2   60.1   42.3   30.2   
upf-forge        with-template [0, 1, 2]   44.74   78.7   54.2   36.9   25.4   
uit-vietnam      with-template [0, 1, 2]   37.97   74.9   54.4   39.5   29.2   

                                                                            \
metric                                    macro_avg_n_stop_words    meteor   
                 subset        references                                    
adaptcentre      with-template [0, 1, 2]                0.337027  0.444883   
melbourne        with-template [0, 1, 2]                0.294784  0.442866   
BIU_nmt          with-template [0, 1, 2]                0.316553  0.443317   
tilburg-smt      with-template [0, 1, 2]                0.281961  0.427105   
abe-1            with-template [0, 1, 2]                0.300239  0.436937   
abe-2            with-template [0, 1, 2]                0.300265  0.436944   
pkuwriter        with-template [0, 1, 2]                0.317959  0.389511   
baseline_sorted  with-template [0, 1, 2]                0.345964  0.379597   
tilburg-nmt      with-template [0, 1, 2]                0.306909  0.416012   
tilburg-template with-template [0, 1, 2]                0.241852  0.391458   
upf-forge        with-template [0, 1, 2]                0.281847  0.417270   
uit-vietnam      with-template [0, 1, 2]                0.242953  0.275285   

                                                     
metric                                          ter  
                 subset        references            
adaptcentre      with-template [0, 1, 2]   0.353064  
melbourne        with-template [0, 1, 2]   0.343148  
BIU_nmt          with-template [0, 1, 2]   0.367139  
tilburg-smt      with-template [0, 1, 2]   0.412763  
abe-1            with-template [0, 1, 2]   0.391338  
abe-2            with-template [0, 1, 2]   0.391338  
pkuwriter        with-template [0, 1, 2]   0.432038  
baseline_sorted